# Graph-Based Orchestration

This example demonstrates graph-based orchestration using pydantic-graph. We build a document quality review loop with typed state and conditional edges.

In [ ]:
from dataclasses import dataclass, field
from pydantic_graph import BaseNode, Graph, GraphRunContext, End
from pydantic import BaseModel
from agentic_patterns.core.agents import get_agent, run_agent

## Define State

The state flows through the graph, accumulating results from each node. Using a typed class makes the data contract explicit.

In [ ]:
@dataclass
class DocumentState:
    topic: str
    draft: str = ""
    score: int = 0
    feedback: str = ""
    revision_count: int = 0
    max_revisions: int = 3

QUALITY_THRESHOLD = 8

## Define Nodes

Each node is a dataclass with a `run` method. The return type annotation determines which nodes can follow.

In [ ]:
class QualityScore(BaseModel):
    score: int
    feedback: str


@dataclass
class GenerateDraft(BaseNode[DocumentState]):
    """Generate initial document draft."""

    async def run(self, ctx: GraphRunContext[DocumentState]) -> "EvaluateQuality":
        agent = get_agent()
        prompt = f"Write a short paragraph (3-4 sentences) about: {ctx.state.topic}"
        result, _ = await run_agent(agent, prompt)
        ctx.state.draft = result.result.output
        print(f"Generated draft:\n{ctx.state.draft}\n")
        return EvaluateQuality()

In [ ]:
@dataclass
class EvaluateQuality(BaseNode[DocumentState]):
    """Evaluate draft quality and decide next step."""

    async def run(self, ctx: GraphRunContext[DocumentState]) -> "Revise | End[str]":
        agent = get_agent(output_type=QualityScore)
        prompt = f"""Evaluate this document on a scale of 1-10 for clarity, completeness, and style.
Provide specific feedback for improvement.

Document:
{ctx.state.draft}"""
        result, _ = await run_agent(agent, prompt)
        ctx.state.score = result.result.output.score
        ctx.state.feedback = result.result.output.feedback
        print(f"Quality score: {ctx.state.score}/10")
        print(f"Feedback: {ctx.state.feedback}\n")

        if ctx.state.score >= QUALITY_THRESHOLD:
            print("Quality threshold met. Finalizing.")
            return End(ctx.state.draft)

        if ctx.state.revision_count >= ctx.state.max_revisions:
            print("Max revisions reached. Finalizing with current draft.")
            return End(ctx.state.draft)

        print("Below threshold. Revising...\n")
        return Revise()

In [ ]:
@dataclass
class Revise(BaseNode[DocumentState]):
    """Revise draft based on feedback."""

    async def run(self, ctx: GraphRunContext[DocumentState]) -> EvaluateQuality:
        ctx.state.revision_count += 1
        agent = get_agent()
        prompt = f"""Improve this document based on the feedback. Return only the improved text.

Current document:
{ctx.state.draft}

Feedback:
{ctx.state.feedback}"""
        result, _ = await run_agent(agent, prompt)
        ctx.state.draft = result.result.output
        print(f"Revision {ctx.state.revision_count}:\n{ctx.state.draft}\n")
        return EvaluateQuality()

## Build the Graph

In [ ]:
graph = Graph(nodes=[GenerateDraft, EvaluateQuality, Revise])

## Visualize the Graph

pydantic-graph can generate mermaid diagrams showing nodes and transitions.

In [ ]:
from IPython.display import Image, display

display(Image(graph.mermaid_image(start_node=GenerateDraft)))

## Run the Graph

In [ ]:
state = DocumentState(topic="The importance of code reviews in software development")

result = await graph.run(GenerateDraft(), state=state)

print("="*50)
print(f"Final document (after {state.revision_count} revisions):\n{result.output}")